# IndicTrans2 HF Inference

We provide an example notebook on how to use our IndicTrans2 models which were originally trained with the fairseq to HuggingFace transformers for inference purpose.


## Setup

Please run the cells below to install the necessary dependencies.


In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

**IMPORTANT : Restart your run-time first and then run the cells below.**

## Inference


In [ ]:
pip install sacrebleu

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [ ]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

### English to Indic Example


In [ ]:
!pip install indic-num2words

In [ ]:
from num_to_words import num_to_word
import re

In [ ]:
pip install accelerate

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [ ]:
import random

relations = ["grandparent", "granddaughter", "grandson", "maternal uncle", "paternal uncle",
             "paternal aunt", "maternal aunt", "nephew", "niece", "cousin", "stepfather",
             "stepmother", "stepsister", "stepbrother", "father-in-law", "mother-in-law",
             "son-in-law", "daughter-in-law", "brother-in-law", "sister-in-law", "grandfather", "grandmother",
             "maternal grandfather", "maternal grandmother",  "sister", "brother",
             "aunt", "paternal grandmother", "paternal grandfather", "mother", "father", "uncle"  ]

subjects = ["I", "She", "He", "They", "My brother", "My sister", "John", "Jane", "Sarah", "Michael"]
verbs = ["visited", "talked to", "baked a cake for", "went to the movies with", "received a gift from",
         "had dinner with", "celebrated birthday with", "helped", "borrowed money from", " argued with", "helped to glam up"]
objects = ["her grandparents", "his grandchildren ", "her maternal uncle", "his paternal aunt",
           "their niece", "her nephew", "my cousins", "his stepsister", "her stepsister","his grandparents", "her grandchildren ",
           "their in-laws", "John's brother-in-law", "John's sister-in-law"]

def generate_sentence():
  subject = random.choice(subjects)
  verb = random.choice(verbs)
  relation = random.choice(relations)
  obj = random.choice(objects)

  # Ensure subject and object agree in number (singular/plural)
  if subject in ["I", "She", "He"]:
    if subject == "I":
      obj = obj.replace("their ", "my ")
    obj = obj.replace(" his", " her").replace(" him", " her")
  else:
    obj= obj.replace("my ", "their ")
    obj = obj.replace(" her", " them").replace(" him", " them")

  sentence = f"{subject} {verb} {obj}"
  return sentence

sentences = []
for i in range(150):
  sentence = generate_sentence()
  sentences.append(sentence)

(sentences)


["Sarah went to the movies with John's sister-in-law",
 'Michael  argued with his paternal aunt',
 "She went to the movies with John's brother-in-law",
 'Jane received a gift from her maternal uncle',
 'Michael baked a cake for their in-laws',
 'Sarah  argued with her grandparents',
 'Michael had dinner with their niece',
 'My brother talked to his paternal aunt',
 "My sister celebrated birthday with John's brother-in-law",
 'She borrowed money from his grandchildren ',
 'My brother received a gift from his stepsister',
 'My sister celebrated birthday with their in-laws',
 'I talked to her grandchildren ',
 'Sarah helped to glam up their niece',
 'He received a gift from her maternal uncle',
 'Sarah helped her maternal uncle',
 'They helped to glam up his stepsister',
 'I helped his grandchildren ',
 'She  argued with her grandchildren ',
 'He had dinner with her grandparents',
 'My brother visited his grandparents',
 'Jane visited their cousins',
 'Jane helped her nephew',
 "Jane help

In [ ]:

en_sents = sentences
#  [

#     #trying out different locations but always gives जाता था in male tone.
#     "When I was young, I used to go to the park every day.",
#         "When I was young, I used to go to the mall every day.",
#     "When I was young, I used to go to the school every day.", # should be vidyalaya and not school??
#         "When I was young, I used to go to the garden every day.",
#         "When I was young, I used to go to the library every day.",
#         "When I was young, I used to go to the parlour every day.",
#             "When I was young, I used to go to the salon every day.",
#             "When I was young, I used to go to the hospital every day.",
#     # trying spl symbols. its ignoring them. DOESN'T convert Numbers to Hindi
#     "*** let's meet at 1:30pm today.",
#  "*** let's meet at 1:30pm today. ***",
# #     #famous tag line "Paytm Karo" could not detect?? - detected but politely
#      "You owe me ₹1234. Paytm me.",
#     "You owe me ₹1234. Do Paytm me. #PaytmKaro",

# #     # numbers in words and digits :: correct translation but showing english numbers instead of hindi numbers
#     "There were only 456 seats left in the auditorium",
# #         "There were only a few hundered seats left in the auditorium",
#     "Almost 3,000 people died in Bhopal gas tragedy",
#     "Almost 3000 people died in Bhopal gas tragedy",
# #         "Almost three thousand people died in Bhopal gas tragedy",

#     "Almost 300 people died in Bahanaga train tragedy",
#     "Almost three hundred people died in Bahanaga train tragedy",
#     # checking if same Noun/ names are preserved
#     "We went to Navi Mumbai",
#     "We went to Mt. Everest",
#     "We ate Aloo Bharta",
#      "We ate Indian mashed potatoes",
#     "We ate rasagulla",
#     "Let's go out for some chaat, I love the spicy flavors.",
#     "Chole is a spicy chickpea curry, perfect with some naan bread.",
#     "Want to eat pani puri?",
#     "Want to eat Pani Puri?", # capital and lower case doesn't matter



#     "I would like to eat Maggie tonight",
#     "Gokuldham is in Goregaon East. I live there.",
#     "Tagline of Indian Tourism is 'Incredible India!' along with the slogan 'Atithi Devo Bhava'.",
#     "Tagline of Telangana is 'It\'s all in it'.",
#      "Tagline of Telangana is It's all in it.",


#     #sound rude but LLM can't
#     "Return my money! Else you are doomed!!!",
# # we need to convert "Indian" english and not actual American native english to Inidc lang



# #can do joint sentences with diff context
#     "He has many old books, which he inherited from his ancestors. I can't figure out how to solve my problem.",

#     #messing with why-Y-y and g00d-good
#         "She is very hardworking and intelligent, which is why she got all the good marks.",
#     "She is very hardworking and inteLLigent, which is y she got all the g00d marks.",

#     "She is very hardworking and inteLLigent, which is Y she got all the g00d marks.",

# # baarish k mausam me pakode khaane k alag hi mazza hai
#     "There is a different pleasure in eating fritters in the rainy season.",# brought from google translate

#     "It's fun to eat fritters in the rainy season.",

#     "We watched a new movie 'Shaitaan' last week, which was very scary.",

#     "If you had met me at that time, we would have gone out to eat.",
#     "She went to the market with her sister to buy a new sari.",
#             #  always paternal
#     "Raj told me that he is going to his grandmother's house next month.", #paternal or maternal??
#         "Rani told me that she is going to her grandmother's house next month.", #paternal or maternal??
#         "They told me that they are going to their grandmother's house next month.", #paternal or maternal??

#     "Raj told me that he is going to his grandfather's house next month.", #paternal or maternal??
#         "Rani told me that she is going to her grandfather's house next month.", #paternal or maternal??
#         "They told me that they are going to their grandfather's house next month.", #paternal or maternal??

#     # covid terms
#     "Kindly follow the SOP for Covid restrictions for the GATE examination at the respective centers.",
#     "Do wear PPE kit while you visit the patients.",
    # "COVID-19 spreads through droplets from coughs/sneezes & touching contaminated surfaces. Caused by the corona virus, it's preventable with masks, distancing & frequent handwashing.",

#     "Kindly follow the SOP for Covid restrictions for GATE at the respective centers.",
#     # all caps
#     "GOOD MORNING!!",
#              "good morning!!",
#              "Good Morning!!",
#              "Good Morning",

#     # toggle case
#     "wE WISH YOU A mERRY cHRISTMAS AND A hAPPY nEW yEAR", #gives different translation than normal sentence

#     "We wish you a Merry Christmas and a Happy New Year",

#     #camel case
#     "Do Visit The Grand Sale At Orion Mall",

#     #smilie emoji
#     "Do Visit The Grand Sale At Orion Mall :)", # translation changed by emoji

    # "I love cats ( ^.^ )<3",
#     "I was surprised on seeing him in the party ( O_O )", # translated O as it is instead of keeping the emoji



#     "My friend has invited me to his birthday party, and I will give him a gift.",

#              #Indian English
#              "He is a topper in his class.",
#              "What is your good name?",
#              "He is out of station.",
#              "I will manage.",
#              "Do you have a pen?",
#              "Leave your footwear outside.",
#              "She is a backbencher.",
#              "Please cooperate.",
#              "He is putting on weight.",
#              "Can I get some water?",
#              "He is on leave.",
#              "What is your programme for today?",
#              "Please have a seat.",
#              "Best of luck for your exams!", #आपकी परीक्षाओं के लिए शुभकामनाएँ!
#              "Please revert back.",
# "I will check and get back to you." , # मैं जाँच करूँगा और आपसे संपर्क करूँगा।
#              "What is your age?",
#              "Please come fast.",
#              "I am feeling hungry.",
#              "He has a cold.",
#              "The train is running late.",
#     "If you go to the market, bring 5kg of potato.",
#     "We are cordially inviting you to grace us with your presence as we, Abhisekh and Sweta, unite in marriage. Let’s create unforgettable memories together on 4th April, 2024.",

# "Ravi is beaming",
#     "British troops fired on a peaceful crowd in Amritsar, killing hundreds of unarmed civilians, sparking outrage and igniting the Indian independence movement.",
#     "The division of British India into India and Pakistan led to mass displacement and violence, leaving a lasting scar on the subcontinent.",
#     "I got the treatment at the Bangalore Baptist Hospital.",
#                  "I got the treatment at the Baptist.",
#              "I got the treatment at the baptist.",
#              "I got the treatment at the Bangalore baptist hospital.",

#     "Let heaven and men and devils, let them all, All, all, cry shame against me, yet I'll speak.", # othello Emilia, Act 5 Scene 2
#     "I got the dress from the Church Street.",#correctly translated
#               "I got the dress from the church street.", # चर्च की सड़क

#              "She will park her cark near the park",
#              "Conquer your work and resist the pull of ego, wrath, lust, attachment, and greed.",
#              "Here lies a reminder of time's power and the inevitability of death.",
#              "Contact us at : my_org@myorg.com",
#              "Please register at : www.some-url.com and subscribe to my YouTube channel."

#  "Its raining cats and dogs."

# ]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
# hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)


# print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")



NameError: name 'hi_translations' is not defined

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(en_sents, hi_translations)), columns=['English', 'Hindi'])

# Define filename
filename = "/content/sample_data/eng_hin_rel.xlsx"
df.to_excel(filename, index=False)

# Open file in write mode and write DataFrame to CSV
# with open(filename, 'w', encoding='utf-8') as csv_file:
#   csv_file.write(df.to_csv( index=False))


In [ ]:
# for text in hi_translations:
#   print(text, "::")
#   numbers = re.findall(r'[\d,]+', text)
#   # translated_numbers=[]
#   for num in numbers:
#     # print("num::", num)
#     if any(char.isdigit() for char in num):

#           # translated_numbers.append(num_to_word(num, lang='hi')) #this hindi needs to be changed TODO
#           # print(translated_numbers)
#   # for i, number in enumerate(translated_numbers):
#   #   text = text.replace(number, translated_numbers[i])
#   # print(text, "\n\n")

#       text = text.replace(num, num_to_word(num, lang='hi'))
#   print(text, "\n\n")


In [ ]:
# for  translation in   hi_translations:
#     # print(f"{src_lang}: {input_sentence}")
#     print(f"'{translation}',")

In [ ]:

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

### Indic to English Example

In [ ]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)

In [ ]:


hi_sents = [
'जब मैं छोटा था, मैं हर दिन पार्क जाता था।',
'जब मैं छोटा था, मैं हर दिन मॉल जाता था।',
'जब मैं छोटा था, मैं हर दिन स्कूल जाता था।',
'जब मैं छोटा था, मैं हर दिन बगीचे में जाता था।',
'जब मैं छोटा था, मैं हर दिन पुस्तकालय जाता था।',
'जब मैं छोटा था, तो मैं हर दिन पार्लर जाता था।',
'जब मैं छोटा था, मैं हर दिन सैलून जाता था।',
'जब मैं छोटा था, तो मैं हर दिन अस्पताल जाता था।',
'चलो आज दोपहर 1ः30 बजे मिलते हैं।',
'चलो आज दोपहर 1ः30 बजे मिलते हैं।',
'आप पर मेरा ₹1234 बकाया है।',
'आप पर मेरा ₹1234 बकाया है। मुझे पेटीएम करें। #PaytmKaro',
'सभागार में केवल 456 सीटें बची थीं।',
'सभागार में केवल कुछ छोटी सीटें बची थीं।',
'भोपाल गैस त्रासदी में लगभग 3,000 लोगों की मौत हो गई',
'भोपाल गैस त्रासदी में लगभग 3000 लोगों की मौत हो गई',
'भोपाल गैस त्रासदी में लगभग तीन हजार लोगों की मौत हो गई',
'बहानगा ट्रेन दुर्घटना में लगभग 300 लोगों की मौत हो गई',
'बहानगा ट्रेन दुर्घटना में लगभग तीन सौ लोगों की मौत हो गई',
'हम नवी मुंबई गए।',
'हम माउंट एवरेस्ट गए।',
'हमने आलू भर्ता खाया।',
'हम भारतीय भुने हुए आलू खाते थे।',
'हम रसगुल्ला खाते थे।',
'चलो कुछ चाट के लिए बाहर जाते हैं, मुझे मसालेदार स्वाद पसंद हैं।',
'छोले एक मसालेदार चने की करी है, जो कुछ नान ब्रेड के साथ एकदम सही है।',
'पानी पुरी खाना चाहते हैं?',
'पानी पुरी खाना चाहते हैं?',
'मैं आज रात मैगी खाना चाहूँगा।',
'गोकुलधाम गोरेगांव पूर्व में है। मैं वहाँ रहता हूँ।',
"भारतीय पर्यटन की टैगलाइन 'अतुल्य भारत!' के साथ-साथ 'अतिथी देवो भव' का नारा है।",
"तेलंगाना की टैगलाइन 'इट्स ऑल इन इट' है।",
'तेलंगाना की टैगलाइन यह है कि इसमें सब कुछ है।',
'मेरे पैसे वापस कर दो! नहीं तो आप बर्बाद हो जाएँगे!!!',
'उनके पास कई पुरानी किताबें हैं, जो उन्हें अपने पूर्वजों से विरासत में मिली हैं। मुझे समझ नहीं आ रहा है कि मैं अपनी समस्या का समाधान कैसे करूं।',
'वह बहुत मेहनती और बुद्धिमान है, यही कारण है कि उसे सभी अच्छे अंक मिले।',
'वह बहुत मेहनती और बुद्धिमान है, यानी उसे सभी जी. आई. डी. अंक मिले हैं।',
'बरसात के मौसम में पराठा खाने का एक अलग आनंद होता है।',
'बरसात के मौसम में पराठा खाने में मजा आता है।',
"हमने पिछले हफ्ते एक नई फिल्म 'शैतान' देखी, जो बहुत डरावनी थी।",
'अगर आप उस समय मुझसे मिलते तो हम बाहर खाना खाने जाते।',
'वह अपनी बहन के साथ नई साड़ी खरीदने के लिए बाजार गई थी।',
'राज ने मुझे बताया कि वह अगले महीने अपनी दादी के घर जा रहा है।',
'रानी ने मुझे बताया कि वह अगले महीने अपनी दादी के घर जा रही है।',
'उन्होंने मुझे बताया कि वे अगले महीने अपनी दादी के घर जा रहे हैं।',
'राज ने मुझे बताया कि वह अगले महीने अपने दादा के घर जा रहा है।',
'रानी ने मुझे बताया कि वह अगले महीने अपने दादा के घर जा रही है।',
'उन्होंने मुझे बताया कि वे अगले महीने अपने दादा के घर जा रहे हैं।',
'कृपया संबंधित केंद्रों पर गेट परीक्षा के लिए कोविड प्रतिबंधों के लिए एस. ओ. पी. का पालन करें।',
'जब आप रोगियों से मिलने जाते हैं तो पी. पी. ई. किट अवश्य पहनें।',
'कोविड-19 खांसने/छींकने और दूषित सतहों को छूने से निकलने वाली बूंदों के माध्यम से फैलता है। कोरोना वायरस के कारण, इसे मास्क, दूरी के साथ रोका जा सकता है और संबंधित केंद्रों पर गेट के लिए कोविड प्रतिबंधों के लिए एस. ओ. पी. का बार-बार पालन करें।',
'अच्छी सुबह!!',
'शुभ प्रभात!',
'शुभ प्रभात!!',
'शुभ प्रभात।',
'हम आपको एक मेरी क्रिसमस और एक खुशहाल नए साल की कामना करते हैं',
'हम आपको क्रिसमस और नए साल की शुभकामनाएँ देते हैं।',
'ओरियन मॉल की ग्रैंड सेल में जरूर जाएँ।',
'ओरियन मॉल में भव्य बिक्री पर अवश्य जाएँः)',
'मुझे बिल्लियाँ पसंद हैं (^. ^) <3',
'मैं उन्हें पार्टी में देखकर हैरान था (O_O)',
'मेरे दोस्त ने मुझे अपने जन्मदिन की पार्टी में आमंत्रित किया है, और मैं उसे एक उपहार दूंगा।',
'वह अपनी कक्षा में टॉपर है।',
'आपका अच्छा नाम क्या है?',
'वह स्टेशन से बाहर है।',
'मैं संभाल लूंगा।',
'क्या आपके पास कलम है?',
'अपने जूतों को बाहर छोड़ दें।',
'वह एक बैकबेंचर हैं।',
'कृपया सहयोग करें।',
'उसका वजन बढ़ रहा है।',
'क्या मुझे थोड़ा पानी मिल सकता है?',
'वह छुट्टी पर है।',
'आज के लिए आपका क्या कार्यक्रम है?',
'कृपया बैठ जाएँ।',
'परीक्षा के लिए आपको बहुत-बहुत बधाई!',
'कृपया वापस लौटें।',
'मैं जाँच करूँगा और आपके पास वापस जाऊँगा।',
'आपकी उम्र क्या है?',
'कृपया जल्दी आ जाएँ।',
'मुझे भूख लग रही है।',
'उसे सर्दी-जुकाम है।',
'ट्रेन देर से चल रही है।',
'बाजार जाएँ तो 5 किलो आलू लाएँ।',
'हम आपको अपनी उपस्थिति के साथ हमें खुश करने के लिए आमंत्रित कर रहे हैं क्योंकि हम, अभिषेक और श्वेता, शादी में एकजुट होते हैं। आइए 4 अप्रैल, 2024 को एक साथ अविस्मरणीय यादें बनाएं।',
'रवि चमक रहा है।',
 'रवि मुस्कुरा रहा है।',
'ब्रिटिश सैनिकों ने अमृतसर में एक शांतिपूर्ण भीड़ पर गोलीबारी की, जिसमें सैकड़ों निहत्थे नागरिक मारे गए, जिससे आक्रोश फैल गया और भारतीय स्वतंत्रता आंदोलन भड़क उठा।',
'भारत और पाकिस्तान में ब्रिटिश भारत के विभाजन ने बड़े पैमाने पर विस्थापन और हिंसा को जन्म दिया, जिससे उपमहाद्वीप पर एक स्थायी निशान रह गया।',
'मेरा इलाज बैंगलोर बैपटिस्ट अस्पताल में हुआ।',
'बैपटिस्ट में मेरा इलाज हुआ।',
'मुझे बैपटिस्ट में इलाज मिला।',
'मेरा इलाज बैंगलोर के बैपटिस्ट अस्पताल में हुआ।',
'स्वर्ग और पुरुषों और शैतानों को, उन सभी को, सभी, सभी, मेरे खिलाफ शर्म से चिल्लाने दें, फिर भी मैं बोलूंगा।',
'मुझे यह पोशाक चर्च स्ट्रीट से मिली थी।',
'मुझे यह पोशाक चर्च की सड़क से मिली।',
'वह अपनी गाड़ी पार्क के पास खड़ी करेगी।',
'मेरा इलाज बैंगलोर बैपटिस्ट अस्पताल में हुआ।',
'स्वर्ग और पुरुषों और शैतानों को, उन सभी को, सभी, सभी, मेरे खिलाफ शर्म से चिल्लाने दें, फिर भी मैं बोलूंगा।',
'मुझे यह पोशाक च्रुच स्ट्रीट से मिली थी।',
'अपने काम पर विजय प्राप्त करें और अहंकार, क्रोध, काम, आसक्ति और लालच के खिंचाव का विरोध करें।',
'यह काल की शक्ति और काल की अनिवार्यता की याद दिलाता है।',
             "फ़ांस मैच जीतने के लिए बहुत सारे टोटके अस्मते है",
             "नानी माँ के लूसके हमशा काम आते हैं"

]
# hi_sents = hi_translations
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)

# for input_sentence, translation in zip(hi_sents, en_translations):
#     print(f"'{input_sentence}',")
    # print(f"{tgt_lang}: {translation}")
print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")



In [ ]:
# flush the models to free the GPU memory
del indic_en_tokenizer, indic_en_model

### Indic to Indic Example


In [ ]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

In [ ]:


hi_sents = [
'जब मैं छोटा था, मैं हर दिन पार्क जाता था।',
'जब मैं छोटा था, मैं हर दिन मॉल जाता था।',
'जब मैं छोटा था, मैं हर दिन स्कूल जाता था।',
'जब मैं छोटा था, मैं हर दिन बगीचे में जाता था।',
'जब मैं छोटा था, मैं हर दिन पुस्तकालय जाता था।',
'जब मैं छोटा था, तो मैं हर दिन पार्लर जाता था।',
'जब मैं छोटा था, मैं हर दिन सैलून जाता था।',
'जब मैं छोटा था, तो मैं हर दिन अस्पताल जाता था।',
'चलो आज दोपहर डेढ़ बजे मिलते हैं।',
'चलो आज दोपहर ढाई बजे मिलते हैं।',
             'चलो आज दोपहर पौने चार बजे मिलते हैं।',
'चलो आज दोपहर पौने चार बजे मिलते हैं।',

'आप पर मेरा ₹१२३४ बकाया है।',
'आप पर मेरा ₹१२३४ बकाया है। मुझे पेटीएम करें। #PaytmKaro',
'सभागार में केवल ४५६ सीटें बची थीं।',
'सभागार में केवल कुछ छोटी सीटें बची थीं।',
'भोपाल गैस त्रासदी में लगभग ३,००० लोगों की मौत हो गई',
'भोपाल गैस त्रासदी में लगभग ३००० लोगों की मौत हो गई',
'भोपाल गैस त्रासदी में लगभग तीन हजार लोगों की मौत हो गई',
'बहानगा ट्रेन दुर्घटना में लगभग ३०० लोगों की मौत हो गई',
'बहानगा ट्रेन दुर्घटना में लगभग तीन सौ लोगों की मौत हो गई',
'हम नवी मुंबई गए।',
'हम माउंट एवरेस्ट गए।',
'हमने आलू भर्ता खाया।',
'हम भारतीय भुने हुए आलू खाते थे।',
'हम रसगुल्ला खाते थे।',
'चलो कुछ चाट के लिए बाहर जाते हैं, मुझे मसालेदार स्वाद पसंद हैं।',
'छोले एक मसालेदार चने की करी है, जो कुछ नान रोटी के साथ एकदम सही है।',
'पानी पुरी खाना चाहते हैं?',
'पानी पुरी खाना चाहते हैं?',
'मैं आज रात मैगी खाना चाहूँगा।',
'गोकुलधाम गोरेगांव पूर्व में है। मैं वहाँ रहता हूँ।',
"भारतीय पर्यटन की टैगलाइन 'अतुल्य भारत!' के साथ-साथ 'अतिथी देवो भव' का नारा है।",
"तेलंगाना की टैगलाइन 'इट्स ऑल इन इट' है।",
'तेलंगाना की टैगलाइन यह है कि इसमें सब कुछ है।',
'मेरे पैसे वापस कर दो! नहीं तो आप बर्बाद हो जाएँगे!!!',
'उनके पास कई पुरानी किताबें हैं, जो उन्हें अपने पूर्वजों से विरासत में मिली हैं। मुझे समझ नहीं आ रहा है कि मैं अपनी समस्या का समाधान कैसे करूं।',
'वह बहुत मेहनती और बुद्धिमान है, यही कारण है कि उसे सभी अच्छे अंक मिले।',
'वह बहुत मेहनती और बुद्धिमान है, यानी उसे सभी जी. आई. डी. अंक मिले हैं।',
'वह बहुत मेहनती और बुद्धिमान है, जो कि Y है, उसे सभी G00d अंक मिले हैं।',
'बरसात के मौसम में पराठा खाने का एक अलग आनंद होता है।',
'बरसात के मौसम में पराठा खाने में मजा आता है।',
"हमने पिछले हफ्ते एक नई फिल्म 'शैतान' देखी, जो बहुत डरावनी थी।",
'अगर आप उस समय मुझसे मिलते तो हम बाहर खाना खाने जाते।',
'वह अपनी बहन के साथ नई साड़ी खरीदने के लिए बाजार गई थी।',
'राज ने मुझे बताया कि वह अगले महीने अपनी दादी के घर जा रहा है।',
'रानी ने मुझे बताया कि वह अगले महीने अपनी दादी के घर जा रही है।',
'उन्होंने मुझे बताया कि वे अगले महीने अपनी दादी के घर जा रहे हैं।',
'राज ने मुझे बताया कि वह अगले महीने अपने दादा के घर जा रहा है।',
'रानी ने मुझे बताया कि वह अगले महीने अपने दादा के घर जा रही है।',
'उन्होंने मुझे बताया कि वे अगले महीने अपने दादा के घर जा रहे हैं।',
'कृपया संबंधित केंद्रों पर गेट परीक्षा के लिए कोविड प्रतिबंधों के लिए एस. ओ. पी. का पालन करें।',
'जब आप रोगियों से मिलने जाते हैं तो पी. पी. ई. किट अवश्य पहनें।',
'कोविड-19 खांसने/छींकने और दूषित सतहों को छूने से निकलने वाली बूंदों के माध्यम से फैलता है। कोरोना वायरस के कारण, इसे मास्क, दूरी के साथ रोका जा सकता है और संबंधित केंद्रों पर गेट के लिए कोविड प्रतिबंधों के लिए एस. ओ. पी. का बार-बार पालन करें।',
'अच्छी सुबह!!',
'शुभ प्रभात!',
'शुभ प्रभात!!',
'शुभ प्रभात।',
'हम आपको एक मेरी क्रिसमस और एक खुशहाल नए साल की कामना करते हैं',
'हम आपको क्रिसमस और नए साल की शुभकामनाएँ देते हैं।',
'ओरियन मॉल की ग्रैंड सेल में जरूर जाएँ।',
'ओरियन मॉल में भव्य बिक्री पर अवश्य जाएँः)',
'मुझे बिल्लियाँ पसंद हैं (^ . ^) <3',
'मैं उन्हें पार्टी में देखकर हैरान था ( O_O )',
'मेरे दोस्त ने मुझे अपने जन्मदिन की पार्टी में आमंत्रित किया है, और मैं उसे एक उपहार दूंगा।',
'वह अपनी कक्षा में टॉपर है।',
'आपका अच्छा नाम क्या है?',
'वह स्टेशन से बाहर है।',
'मैं संभाल लूंगा।',
'क्या आपके पास कलम है?',
'अपने जूतों को बाहर छोड़ दें।',
'वह एक बैकबेंचर हैं।',
'कृपया सहयोग करें।',
'उसका वजन बढ़ रहा है।',
'क्या मुझे थोड़ा पानी मिल सकता है?',
'वह छुट्टी पर है।',
'आज के लिए आपका क्या कार्यक्रम है?',
'कृपया बैठ जाएँ।',
'परीक्षा के लिए आपको बहुत-बहुत बधाई!',
'कृपया वापस लौटें।',
'मैं जाँच करूँगा और आपके पास वापस जाऊँगा।',
'आपकी उम्र क्या है?',
'कृपया जल्दी आ जाएँ।',
'मुझे भूख लग रही है।',
'उसे सर्दी-जुकाम है।',
'ट्रेन देर से चल रही है।',
'बाजार जाएँ तो 5 किलो आलू लाएँ।',
'हम आपको अपनी उपस्थिति के साथ हमें खुश करने के लिए आमंत्रित कर रहे हैं क्योंकि हम, अभिषेक और श्वेता, शादी में एकजुट होते हैं। आइए 4 अप्रैल, 2024 को एक साथ अविस्मरणीय यादें बनाएं।',
'रवि चमक रहा है।',
 'रवि मुस्कुरा रहा है।',
'ब्रिटिश सैनिकों ने अमृतसर में एक शांतिपूर्ण भीड़ पर गोलीबारी की, जिसमें सैकड़ों निहत्थे नागरिक मारे गए, जिससे आक्रोश फैल गया और भारतीय स्वतंत्रता आंदोलन भड़क उठा।',
'भारत और पाकिस्तान में ब्रिटिश भारत के विभाजन ने बड़े पैमाने पर विस्थापन और हिंसा को जन्म दिया, जिससे उपमहाद्वीप पर एक स्थायी निशान रह गया।',
'मेरा इलाज बैंगलोर बैपटिस्ट अस्पताल में हुआ।',
'बैपटिस्ट में मेरा इलाज हुआ।',
'मुझे बैपटिस्ट में इलाज मिला।',
'मेरा इलाज बैंगलोर के बैपटिस्ट अस्पताल में हुआ।',
'स्वर्ग और पुरुषों और शैतानों को, उन सभी को, सभी, सभी, मेरे खिलाफ शर्म से चिल्लाने दें, फिर भी मैं बोलूंगा।',
'मुझे यह पोशाक चर्च स्ट्रीट से मिली थी।',
'मुझे यह पोशाक चर्च की सड़क से मिली।',
'वह अपनी गाड़ी पार्क के पास खड़ी करेगी।',
'मेरा इलाज बैंगलोर बैपटिस्ट अस्पताल में हुआ।',
'स्वर्ग और पुरुषों और शैतानों को, उन सभी को, सभी, सभी, मेरे खिलाफ शर्म से चिल्लाने दें, फिर भी मैं बोलूंगा।',
'मुझे यह पोशाक च्रुच स्ट्रीट से मिली थी।',
'अपने काम पर विजय प्राप्त करें और अहंकार, क्रोध, काम, आसक्ति और लालच के खिंचाव का विरोध करें।',
'यह काल की शक्ति और काल की अनिवार्यता की याद दिलाता है।',
             "फ़ांस मैच जीतने के लिए बहुत सारे टोटके अस्मते है",
             "नानी माँ के लूसके हमेशा काम आते हैं",
             'जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल',
    'मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है',
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
    "आप पर मेरा ₹1234 बकाया है।  पेटीएम करें।",
    "वह समुद्र तट पर समुद्री गोले बेचती है।",
    "मेरे चचेरे भाइयों ने मेरे भाई की शादी में नृत्य किया",
    "मेरे भाभी को चॉकलेट पसंद है",
    "मेरे भाई को चॉकलेट खाना बहुत पसंद है।",
    "मेरे मामा की जूतों की दुकान है।",
    "मेरी नानी हमारे लिए स्वादिष्ट खाना बनाती हैं।",
    "मेरी ताई जी रोज जिम जाती हैं।",
    "मेरे जीजा ने मेहमानों के लिए जगह साफ करने में मेरी मदद की।",
    "मेरी बड़ी माँ ने मेहमानों के लिए जगह साफ करने में मेरी मदद की",
    "मेरी दादी हमारे लिए स्वादिष्ट भोजन बनाती हैं।",
    "मेरे दादा एक बुद्धिमान व्यक्ति हैं।",
    "हम कल अपनी दादी के घर गए थे।",
    "हम आज अपनी चाची के पास जाएँगे।",
    "मेरी नीस इतनी प्यारी है कि जब भी मैं उसे देखता हूं तो मैं उसका चेहरा निचोड़ना चाहता हूं।",
    "मेरे भतीजे ने पिछले हफ्ते स्कूल जाना शुरू कर दिया।"
    "यह बच्चा मेरी आँखों का तारा है",
    "उसकी क्या बात कर रहे हो, वह तो मेरे खून का प्यासा हो गया है।",
    "आतंकवादियों को देखकर मेरा तो खून ठण्डा पड़ गया।",
    "आई.पी.एस पास करके दीक्षा ने सचमुच गढ़ फतह कर लिया।",
    "कार्तिक गधे को बाप बनाना अच्छी तरह जानता हैं।",
    "अभी आपको मेरी बात समझ में नहीं आ रही, जब जूते पड़ेंगे तब समझ में आएगी।",
    "जहाँ चाह, वहाँ राह।",
    "हर कड़ी में राह होती है।",
    "काम का बोझ भारी होता है।",
    "रोम एक दिन में नहीं बना था।",
    "अब पछताए होत क्या जब चिड़ीया चुग गई खेत।",
    "हर चमकते चीज सोना नहीं होती।",
    "दो गलत सही नहीं बनाते।",
    "जहाँ धूम है, वहाँ आग होगी।",
    "विद्यार्थी स्कूल जाने का समय है। यह बहुत महत्वपूर्ण है कि आप समय पर स्कूल पहुंचें ताकि आप अपने पढ़ाई में निरंतरता बनाए रख सकें। स्कूल जाना आपके शिक्षा के लिए आवश्यक है और यह आपके भविष्य को सुनिश्चित करने में मदद कर सकता है। इसलिए, आप आज के दिन को सकारात्मकता के साथ शुरू करें और स्कूल के लिए समय पर निकलें। शुभ कामनाएं!",
    "शुभ प्रभात! आपका दिन सुखमय और सफल हो",
    "दिव्यांग व्यक्तियों की मदद करना सामाजिक जिम्मेदारी है।",
    "इन व्यक्तियों को समाज में समाहित करने और उन्हें समर्थ बनाने के लिए हमें सकारात्मक कदम उठाने चाहिए",
    "दिव्यांग व्यक्तियों के साथ सहानुभूति और समर्थन बनाए रखना अत्यंत महत्वपूर्ण है।",
    "उन्हें समाज में पूरी तरह से शामिल करना और उनकी सामाजिक समाहिति को बढ़ावा देना।",
    "उन्हें उच्च शिक्षा और प्रशिक्षण के अवसरों का भी लाभ दिलाना।",
    "सरकार और सामाजिक संगठनों से उनके लिए सुविधाएं और योजनाएं मांगना और उन्हें इनका लाभ उठाने में मदद करना।",
    "उन्हें रोजगार के अवसरों को सुझाना और उन्हें समर्थ बनाने के लिए तकनीकी और व्यावासायिक प्रशिक्षण प्रदान करना।",
    "सामाजिक संगठनों में सक्रियता दिखाना और दिव्यांगता के मुद्दों पर जागरूकता फैलाना।",
    "यह सभी कदम सामूहिक रूप से समृद्धि और समाज में समाहिति की दिशा में सहायक हो सकते हैं।",
    "कनक कनक ते सौ गुनी, मादकता अधिकाय। वा खाए बौराय जग, या पाए बौराय।",
    "उसकी मुस्कान सूर्य की किरणों की तरह चमक रही थी।",
    "समय एक अजीब सा दरवाज़ा है, जो कभी फिर नहीं खुलता।",
    "बादल बढ़ रहे हैं, बारिश बरस रही है।",
    "बस बुढ़िया बैठी बाल कांपा कर बोली।",
    "बाग़ के फूलों ने रात को महका दिया।",
    "गंगा जी की धारा, स्वर्ग से भी प्यारी है।",
    "सुरमा स्वर्ग से सजीवनी लेकर आई।",
    "उसकी आँखों में आसमान का रंग और उसके बोल में दरिया की गहराई थी",
    "हनुमान की पूंछ में लगन ना पाई आग, लंका सारी जल गई, गए निशाचर भग"

]
src_lang, tgt_lang = "hin_Deva", "ory_Orya"
mr_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, mr_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


In [ ]:
# for input_sentence, translation in zip(hi_sents, mr_translations):
#     # print(f"{src_lang}: {input_sentence}")
#     print(f"'{translation}',")

In [ ]:


or_sents = [
    'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ପାର୍କକୁ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ମଲ୍କୁ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ବିଦ୍ୟାଳୟ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ବଗିଚାକୁ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ପାଠାଗାରକୁ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ପାର୍ଲର୍କୁ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ସେଲୁନ ଯାଉଥିଲି।',
'ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ଡାକ୍ତରଖାନାକୁ ଯାଉଥିଲି।',
'ଆଜି ଅପରାହ୍ନ ଦେଢ଼ଟାରେ ଭେଟିବା।',
'ଚାଲନ୍ତୁ ଆଜି ଅପରାହ୍ନ ଚାରିଟାରେ ଭେଟିବା।',
'ଚାଲନ୍ତୁ ଆଜି ଅପରାହ୍ନ ଚାରିଟାରେ ଭେଟିବା।',
'ଆପଣମାନେ ମୋ ଉପରେ ୧୨୩୪ ଟଙ୍କା ବକେଯ଼ା ରଖିଛନ୍ତି।',
'ମୋ ଉପରେ ଆପଣଙ୍କର ୧୨୩୪ ଟଙ୍କା ବକେଯ଼ା ଅଛି। ମୋତେ ପେଟିଏମ୍ କରନ୍ତୁ। #PaytmKaro',
'ସଭାଗାରରେ କେବଳ ୪୫୬ଟି ଆସନ ବାକି ଥିଲା।',
'ଅଡିଟୋରିଯ଼ମରେ କେବଳ କିଛି ଛୋଟ ଆସନ ବାକି ଥିଲା।',
'ଭୋପାଳ ଗ୍ଯ଼ାସ୍ ଦୁର୍ଘଟଣାରେ ପ୍ରାଯ଼ ୩,୦୦୦ ଲୋକ ପ୍ରାଣ ହରାଇଛନ୍ତି।',
'ଭୋପାଳ ଗ୍ଯ଼ାସ୍ ଦୁର୍ଘଟଣାରେ ପ୍ରାଯ଼ ୩୦୦୦ ଲୋକ ପ୍ରାଣ ହରାଇଛନ୍ତି।',
'ଭୋପାଳ ଗ୍ଯ଼ାସ୍ ଦୁର୍ଘଟଣାରେ ପ୍ରାଯ଼ ତିନି ହଜାର ଲୋକ ପ୍ରାଣ ହରାଇଛନ୍ତି।',
'ବାହାନାଙ୍ଗା ଟ୍ରେନ୍ ଦୁର୍ଘଟଣାରେ ପ୍ରାଯ଼ ୩୦୦ ଜଣଙ୍କର ମୃତ୍ଯ଼ୁ',
'ବାହାନାଙ୍ଗା ଟ୍ରେନ୍ ଦୁର୍ଘଟଣାରେ ପ୍ରାଯ଼ ତିନି ଶହ ଲୋକ ପ୍ରାଣ ହରାଇଥିଲେ।',
'ଆମେ ନାଭି ମୁମ୍ବାଇ ଗଲୁ।',
'ଆମେ ମାଉଣ୍ଟ ଏଭରେଷ୍ଟ ଯାଇଥିଲୁ।',
'ଆମେ ଆଳୁ ଭର୍ତା ଖାଇଲୁ।',
'ଆମେ ରସଗୋଲା ଖାଉଥିଲୁ।',
'ଚାଲ ଚାଟ୍ ପାଇଁ ବାହାରକୁ ଯିବା, ମୁଁ ମସଲାଯୁକ୍ତ ସ୍ୱାଦ ପସନ୍ଦ କରେ |',
'ଚୋଲେ ହେଉଛି ଏକ ମସଲାଯୁକ୍ତ ଚଣା ତରକାରୀ, ଯାହା କିଛି ନାନ ରୁଟି ସହିତ ଉପଯୁକ୍ତ |',
'ପାଣି ପୁରୀ ଖାଇବାକୁ ଚାହୁଁଛନ୍ତି କି?',
'ମୁଁ ଆଜି ରାତିରେ ମ୍ଯ଼ାଗି ଖାଇବାକୁ ଚାହୁଁଛି।',
'ଗୋରେଗାଓଁ ଇଷ୍ଟରେ ଗୋକୁଳଧାମ ଅଛି। ମୁଁ ସେଠାରେ ରୁହେ।',
"ଭାରତୀଯ଼ ପର୍ଯ୍ଯ଼ଟନର ଟ୍ଯ଼ାଗଲାଇନ ହେଉଛି 'ଅତୁଳନୀୟ ଭାରତ!'ଏବଂ ଏହାର ସ୍ଲୋଗାନ ହେଉଛି 'ଅତିଥୀ ଦେବ ଭବ'।",
"ତେଲେଙ୍ଗାନାର ଟ୍ଯ଼ାଗଲାଇନ ହେଉଛି 'ଇଟ୍ସ୍ ଅଲ୍ ଇନ୍ ଇଟ୍'।",
'ତେଲେଙ୍ଗାନାର ଟ୍ଯ଼ାଗଲାଇନ ହେଉଛି ଯେ ଏଥିରେ ସବୁକିଛି ଅଛି।',
'ମୋ ଟଙ୍କା ଫେରସ୍ତ କର! ନଚେତ୍ ତୁମେ ନଷ୍ଟ ହୋଇଯିବ!!!',
'ତାଙ୍କ ପାଖରେ ଅନେକ ପୁରୁଣା ପୁସ୍ତକ ଅଛି, ଯାହା ସେ ତାଙ୍କ ପୂର୍ବପୁରୁଷଙ୍କଠାରୁ ଉତ୍ତରାଧିକାରରେ ପାଇଛନ୍ତି। ମୁଁ ବୁଝିପାରୁନାହିଁ ଯେ ମୁଁ ମୋର ସମସ୍ଯ଼ାକୁ କିପରି ସମାଧାନ କରିବି।',
'ସେ ବହୁତ ପରିଶ୍ରମୀ ଏବଂ ବୁଦ୍ଧିମାନ, ସେଥିପାଇଁ ସେ ସମସ୍ତ ଭଲ ଅଙ୍କ ପାଇଥିଲେ।',
'ସେ ବହୁତ ପରିଶ୍ରମୀ ଏବଂ ବୁଦ୍ଧିମାନ, ଯାହା ୱାଇ, ତାଙ୍କର ସମସ୍ତ ଜି00ଡି ଅଙ୍କ ଅଛି।',
'ବର୍ଷା ଋତୁରେ ପକୁଡି ଖାଇବା ଏକ ଭିନ୍ନ ଆନନ୍ଦ।',
"ଆମେ ଗତ ସପ୍ତାହରେ 'ଶୈତାନ' ନାମକ ଏକ ନୂଆ ଚଳଚ୍ଚିତ୍ର ଦେଖିଲୁ, ଯାହା ଅତ୍ଯ଼ନ୍ତ ଭଯ଼ଙ୍କର ଥିଲା।",
'ଯଦି ଆପଣ ମୋତେ ସେହି ସମଯ଼ରେ ଭେଟିଥାନ୍ତେ, ତେବେ ଆମେ ରାତ୍ରିଭୋଜନ ପାଇଁ ବାହାରକୁ ଯାଇଥାନ୍ତେ',
'ସେ ତାଙ୍କ ଦିଦିଙ୍କ ସହ ନୂଆ ଶାଢ଼ୀ କିଣିବା ପାଇଁ ବଜାରକୁ ଯାଇଥିଲେ।',
"ରାଜ ମୋତେ କହିଥିଲେ ଯେ ସେ ଆସନ୍ତା ମାସରେ ତାଙ୍କ ଜେଜେମା 'ଙ୍କ ଘରକୁ ଯାଉଛନ୍ତି।",
"ରାଣୀ ମୋତେ କହିଥିଲେ ଯେ ସେ ଆସନ୍ତା ମାସରେ ତାଙ୍କ ଜେଜେମା 'ଙ୍କ ଘରକୁ ଯାଉଛନ୍ତି।",
"ସେମାନେ ମୋତେ କହିଥିଲେ ଯେ ସେମାନେ ଆସନ୍ତା ମାସରେ ସେମାନଙ୍କ ଆଈ'ଙ୍କ ଘରକୁ ଯାଉଛନ୍ତି।",
'ରାଜ ମୋତେ କହିଥିଲେ ଯେ ସେ ଆସନ୍ତା ମାସରେ ତାଙ୍କ ଅଜାଙ୍କ ଘରକୁ ଯାଉଛନ୍ତି।',
'ରାଣୀ ମୋତେ କହିଥିଲେ ଯେ ସେ ଆସନ୍ତା ମାସରେ ତାଙ୍କ ଜେଜେବାପାଙ୍କ ଘରକୁ ଯାଉଛନ୍ତି।',
'ସେମାନେ ମୋତେ କହିଥିଲେ ଯେ ସେମାନେ ଆସନ୍ତା ମାସରେ ସେମାନଙ୍କ ଜେଜେବାପାଙ୍କ ଘରକୁ ଯାଉଛନ୍ତି।',
'ଦଯ଼ାକରି ସମ୍ପୃକ୍ତ କେନ୍ଦ୍ରରେ ଗେଟ୍ ପରୀକ୍ଷା ପାଇଁ କୋଭିଡ ପ୍ରତିବନ୍ଧକ ପାଇଁ ଏସ୍. ଓ. ପି. ଅନୁସରଣ କରନ୍ତୁ।',
'ରୋଗୀମାନଙ୍କୁ ଭେଟିବା ସମଯ଼ରେ ପି. ପି. ଇ. କିଟ୍ ପିନ୍ଧିବା ନିଶ୍ଚିତ କରନ୍ତୁ।',
'କାଶ/ଛିଙ୍କିବା ଏବଂ ଦୂଷିତ ପୃଷ୍ଠକୁ ଛୁଇଁବା ଦ୍ୱାରା ବାହାରିଥିବା ବୁନ୍ଦା ମାଧ୍ଯ଼ମରେ COVID-19 ବ୍ଯ଼ାପିଥାଏ | କରୋନଭାଇରସ୍ ହେତୁ, ଏହାକୁ ମାସ୍କ, ଦୂରତା ସହିତ ରୋକାଯାଇପାରିବ ଏବଂ ସମ୍ପୃକ୍ତ କେନ୍ଦ୍ରରେ ଗେଟ୍ ପାଇଁ COVID ପ୍ରତିବନ୍ଧକ ପାଇଁ<unk> କୁ ବାରମ୍ବାର ଅନୁସରଣ କରାଯାଇପାରିବ |',
'ଶୁଭ ସକାଳ!!',
'ସୁପ୍ରଭାତ',
'ଆମେ ଆପଣଙ୍କୁ ଖ୍ରୀଷ୍ଟମାସ ଏବଂ ନୂତନ ବର୍ଷର ଶୁଭେଚ୍ଛା ଜଣାଉଛୁ।',
'ଓରିଅନ୍ ମଲ୍ର ଗ୍ରାଣ୍ଡ ସେଲ୍କୁ ନିଶ୍ଚଯ଼ ଯାଆନ୍ତୁ।',
'ମୁଁ ବିଲେଇମାନଙ୍କୁ ଭଲ ପାଏ ( ^.^ ) <3',
'ପାର୍ଟିରେ ତାଙ୍କୁ ଦେଖି ମୁଁ ଆଶ୍ଚର୍ଯ୍ଯ଼ ହୋଇଯାଇଥିଲି ( O_O )',
'ମୋ ବନ୍ଧୁ ମୋତେ ତାଙ୍କ ଜନ୍ମଦିନ ପାର୍ଟିକୁ ନିମନ୍ତ୍ରଣ କରିଛନ୍ତି, ଏବଂ ମୁଁ ତାଙ୍କୁ ଏକ ଉପହାର ଦେବି।',
'ସେ ତାଙ୍କ ଶ୍ରେଣୀରେ ଟପ୍ପର ଅଟନ୍ତି।',
'ଆପଣଙ୍କ ଭଲ ନାମ କ "ଣ?',
'ସେ ଷ୍ଟେସନ ବାହାରେ ଅଛନ୍ତି।',
'ମୁଁ ଏହାର ଯତ୍ନ ନେବି।',
'ଆପଣଙ୍କ ପାଖରେ କଲମ ଅଛି କି?',
'ଜୋତା ଗୁଡ଼ିକୁ ବାହାରେ ଛାଡ଼ି ଦିଅନ୍ତୁ।',
'ସେ ଜଣେ ବ୍ଯ଼ାକ୍ବେଞ୍ଚର୍।',
'ଦଯ଼ାକରି ସହଯୋଗ କରନ୍ତୁ।',
'ତାଙ୍କର ଓଜନ ବଢ଼ୁଛି।',
'ମୁଁ କିଛି ପାଣି ପାଇପାରିବି କି?',
'ସେ ଛୁଟିରେ ଅଛନ୍ତି।',
'ଆଜି ପାଇଁ ଆପଣଙ୍କ କାର୍ଯ୍ଯ଼ସୂଚୀ କ "ଣ?',
'ଦଯ଼ାକରି ବସନ୍ତୁ।',
'ଏହି ପରୀକ୍ଷା ପାଇଁ ଆପଣଙ୍କୁ ବହୁତ ବହୁତ ଅଭିନନ୍ଦନ!',
'ଦଯ଼ାକରି ଫେରିଯାଆନ୍ତୁ।',
'ମୁଁ ଯାଞ୍ଚ କରିବି ଏବଂ ତୁମ ପାଖକୁ ଫେରିବି।',
'ଆପଣଙ୍କ ବଯ଼ସ କେତେ?',
'ଦଯ଼ାକରି ଶୀଘ୍ର ପହଞ୍ଚନ୍ତୁ।',
'ମୁଁ ଭୋକିଲା ଅଛି।',
'ତାଙ୍କୁ ଥଣ୍ଡା ଅଛି।',
'ଟ୍ରେନ୍ ବିଳମ୍ବରେ ଚାଲୁଛି।',
'ଯେତେବେଳେ ଆପଣ ବଜାରକୁ ଯାଆନ୍ତି, ୫ କିଲୋ ଆଳୁ ଆଣନ୍ତୁ।',
'ଆମେ ଆପଣଙ୍କୁ ଆମ ଉପସ୍ଥିତି ଦ୍ୱାରା ଆମକୁ ଖୁସି କରିବା ପାଇଁ ନିମନ୍ତ୍ରଣ କରୁଛୁ କାରଣ ଆମେ, ଅଭିଷେକ ଏବଂ ଶ୍ୱେତା, ବିବାହରେ ଏକାଠି ଅଛୁ | ଆସନ୍ତୁ ଏପ୍ରିଲ୍ 4,2024 ରେ ଏକାଠି ଅବିସ୍ମରଣୀଯ଼ ସ୍ମୃତି ସୃଷ୍ଟି କରିବା |',
'ରବି ଚମକି ପଡ଼ୁଛି।',
'ରବି ହସୁଛି।',
'ବ୍ରିଟିଶ ସୈନିକମାନେ ଅମୃତସରରେ ଏକ ଶାନ୍ତିପୂର୍ଣ୍ଣ ଭିଡ଼ ଉପରେ ଗୁଳି ଚଳାଇଥିଲେ, ଯେଉଁଥିରେ ଶହ ଶହ ନିରସ୍ତ୍ର ନାଗରିକଙ୍କ ମୃତ୍ଯ଼ୁ ଘଟିଥିଲା, ଯାହା କ୍ରୋଧ ସୃଷ୍ଟି କରିଥିଲା ଏବଂ ଭାରତୀଯ଼ ସ୍ୱାଧୀନତା ଆନ୍ଦୋଳନକୁ ଉତ୍ତେଜିତ କରିଥିଲା।',
'ଭାରତ ଏବଂ ପାକିସ୍ତାନରେ ବ୍ରିଟିଶ ଭାରତର ବିଭାଜନ ବ୍ଯ଼ାପକ ବିସ୍ଥାପନ ଏବଂ ହିଂସାକୁ ଜନ୍ମ ଦେଇଥିଲା, ଯାହା ଉପମହାଦେଶରେ ଏକ ସ୍ଥାଯ଼ୀ ଚିହ୍ନ ଛାଡିଥିଲା।',
'ବାଙ୍ଗାଲୋର ବାପ୍ଟିଷ୍ଟ ହସ୍ପିଟାଲରେ ମୋର ଚିକିତ୍ସା କରାଯାଇଥିଲା।',
'ବାପ୍ଟିଷ୍ଟଠାରେ ମୋର ଚିକିତ୍ସା କରାଯାଇଥିଲା।',
'ମୁଁ ବାପ୍ଟିଷ୍ଟରେ ଚିକିତ୍ସା ପାଇଲି।',
'ବାଙ୍ଗାଲୋରର ବାପ୍ଟିଷ୍ଟ ହସ୍ପିଟାଲରେ ମୋର ଚିକିତ୍ସା କରାଯାଇଥିଲା।',
'ସ୍ୱର୍ଗ ଏବଂ ମନୁଷ୍ଯ଼ ଏବଂ ଶଯ଼ତାନ, ସେମାନେ ସମସ୍ତେ, ସମସ୍ତେ, ମୋ ବିରୁଦ୍ଧରେ ଲଜ୍ଜାରେ ଚିତ୍କାର କରନ୍ତୁ, ତଥାପି ମୁଁ କହିବି।',
'ମୁଁ ଏହି ପୋଷାକ ଚର୍ଚ୍ଚ ଷ୍ଟ୍ରିଟ୍ ରୁ ପାଇଲି।',
'ସେ ତାଙ୍କ କାରକୁ ପାର୍କ ନିକଟରେ ପାର୍କିଂ କରିବେ।',
'ଆପଣଙ୍କ କାର୍ଯ୍ଯ଼କୁ ଅତିକ୍ରମ କରନ୍ତୁ ଏବଂ ଅହଂକାର, କ୍ରୋଧ, କାମ, ଆସକ୍ତି ଏବଂ ଲୋଭର ଆକର୍ଷଣକୁ ପ୍ରତିରୋଧ କରନ୍ତୁ।',
'ଏହା କାଳର ଶକ୍ତି ଏବଂ କାଳର ଅପରିହାର୍ଯ୍ଯ଼ତାର ଏକ ଅନୁସ୍ମାରକ।',
'ପ୍ରଶଂସକ ମ୍ଯ଼ାଚ୍ ଜିତିବା ପାଇଁ ଅନେକ କୌଶଳ ଅଛି |',

]
src_lang, tgt_lang = "ory_Orya","hin_Deva"
hi_translations = batch_translate(or_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(or_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


In [ ]:

# flush the models to free the GPU memory
del indic_indic_tokenizer, indic_indic_model